In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import data_util  
yf.pdr_override()
from sklearn.svm import SVC

sns.set(rc={'figure.figsize':(20, 15)})

temp = dict(layout=go.Layout(font=dict(family="Franklin Gothic", size=12), width=800))

import warnings
warnings.filterwarnings('ignore')


In [2]:
from Evaluation_metrics import *
def evaluate_metrics(y_true, y_pred):
    results = {
        'Metric': ['MAPE', 'MAE', 'MSE', 'RMSE', 'R2', 'SMAPE'],
        'Value': [MAPE(y_true, y_pred), 
                  MAE(y_true, y_pred), 
                  MSE(y_true, y_pred), 
                  RMSE(y_true, y_pred), 
                  R2(y_true, y_pred), 
                  SMAPE(y_true, y_pred)]
    }
    df = pd.DataFrame(results)
    
    # Add dotted lines
    dotted_style = [dict(selector="th", props=[("border-bottom", "1px dotted #aaaaaa")]),
                    dict(selector="td", props=[("border-bottom", "1px dotted #aaaaaa")])]
    
    # Apply styling
    styled_df = (df.style
                 .set_properties(**{'text-align': 'center'})
                 .format({'Value': '{:.2f}'})  # Round values to 2 decimal places
                 .set_table_styles(dotted_style)
                 .set_caption('Evaluation Metrics')
                 .set_table_attributes('style="border-collapse: collapse; border: none;"')
                 .set_properties(subset=['Metric'], **{'font-weight': 'bold',}))  # Bold and blue headers
    
    return styled_df

In [3]:
stocks=data_util.stock_data('^NSEI', 4,'Nifty 50')

[*********************100%%**********************]  1 of 1 completed


In [4]:
from prophet import Prophet
from prophet.plot import plot_plotly

# Assuming you have already imported necessary libraries and loaded the data

# Reset index of stocks DataFrame
stocks.reset_index(inplace=True)
# stocks.set_index('Date',inplace=True)
stocks_temp = stocks[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
split=len(stocks_temp)-int(len(stocks_temp)*0.70)

stocks_temp_test = stocks_temp.tail(split)
print(split)

stocks_temp_train = stocks_temp.head(len(stocks_temp) - split)

FBprophet = Prophet(daily_seasonality=True)

FBprophet.fit(stocks_temp_train)

future = FBprophet.make_future_dataframe(periods=split)

forecast = FBprophet.predict(future)

fig = plot_plotly(FBprophet, forecast)

fig.add_trace(go.Scatter(x=stocks_temp_test['ds'], y=stocks_temp_test['y'], mode='markers', name='Test Data'))

fig.show()



297


06:28:18 - cmdstanpy - INFO - Chain [1] start processing
06:28:18 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
y_pred=forecast["yhat"].values[int(len(stocks_temp)*0.70):]
y_true=stocks_temp['y'].values[int(len(stocks_temp)*0.70):]
styled_df = evaluate_metrics(y_true, y_pred)
styled_df

,Metric,Value
0,MAPE,9.65
1,MAE,1994.14
2,MSE,5095197.24
3,RMSE,2257.25
4,R2,-0.84
5,SMAPE,10.26
